# Imports

In [2]:
import numpy as np
import pandas as pd
import sys

from torch.utils import data

sys.path.append("../../src")
sys.path.append("../palm-tree")

import embedder
import models
import utils
import dataset

# Data

In [3]:
basic_block_a = [
    "mov rbp rdi",
    "mov ebx 0x1",
    "mov rdx rbx",
    "call memcpy",
    "mov [ rcx + rbx ] 0x0",
    "mov rcx rax",
    "mov [ rax ] 0x2e"
]

basic_block_b = [
    "mov rbp rdi",
    "mov ebx 0x1",
    "mov rdx rbx",
    "call memcpy",
    "add     DWORD PTR [rbp-4], eax",
    "mov     eax, DWORD PTR [rbp-4]",
    "add     DWORD PTR [rbp-8], eax",
    "mov [ rcx + rbx ] 0x0",
    "mov rcx rax",
    "mov [ rax ] 0x2e"
]

basic_block_c = [
    "push    rbp",
    "mov     rbp, rsp",
    "mov     DWORD PTR [rbp-4], 1",
    "mov     DWORD PTR [rbp-8], 2",
    "mov     eax, DWORD PTR [rbp-8]",
    "add     DWORD PTR [rbp-4], eax",
    "mov     eax, DWORD PTR [rbp-4]",
    "add     DWORD PTR [rbp-8], eax",
    "mov     eax, 0",
    "pop     rbp"
]

blocks =  [basic_block_a, basic_block_b, basic_block_c]

In [4]:
embeddings = [embedder.encode(bb) for bb in blocks]

Loading Vocab C:/Users/thodo/Documents/σχολη/diplomatiki/PalmTree/pre-trained_model/palmtree/vocab
Vocab Size:  6631
Loading Vocab C:/Users/thodo/Documents/σχολη/diplomatiki/PalmTree/pre-trained_model/palmtree/vocab
Vocab Size:  6631
Loading Vocab C:/Users/thodo/Documents/σχολη/diplomatiki/PalmTree/pre-trained_model/palmtree/vocab
Vocab Size:  6631


In [12]:
random_labels = np.random.randint(100, size=len(embeddings))
random_labels

array([25,  6, 10])

In [13]:
data_df = pd.DataFrame({"blocks": blocks, "embeddings": embeddings, "energy": random_labels})

In [14]:
data_df

,blocks,embeddings,energy
0,"[mov rbp rdi, mov ebx 0x1, mov rdx rbx, call m...","[[2.1426268, -0.83879894, 0.1582396, -2.787814...",25
1,"[mov rbp rdi, mov ebx 0x1, mov rdx rbx, call m...","[[2.1426268, -0.83879894, 0.1582396, -2.787814...",6
2,"[push rbp, mov rbp, rsp, mov DWORD ...","[[-1.0875186, -3.307008, -3.174919, 2.858612, ...",10


In [17]:
ds = dataset.EnergyPredictionDataset(data_df)
data_loader = data.DataLoader(ds, batch_size=2, shuffle=False, collate_fn=utils.collate_fn, drop_last=True)

In [18]:
batch = next(iter(data_loader))
batch

(tensor([[[ 2.1426, -0.8388,  0.1582,  ...,  2.6131, -0.0776,  1.7005],
          [ 0.0357,  1.6601, -2.2026,  ...,  4.2156,  0.6422, -2.5731],
          [-1.4210,  0.3558, -0.6415,  ...,  4.7222, -0.2358,  2.5334],
          ...,
          [ 2.0509,  0.1768, -1.3684,  ...,  0.7584, -0.9806,  0.4287],
          [ 2.4320, -0.4087,  1.9237,  ...,  6.5683, -0.0989, -0.5761],
          [ 2.4185, -4.6341,  1.8846,  ...,  1.1307, -0.6543,  0.0513]],
 
         [[ 2.1426, -0.8388,  0.1582,  ...,  2.6131, -0.0776,  1.7005],
          [ 0.0357,  1.6601, -2.2026,  ...,  4.2156,  0.6422, -2.5731],
          [-1.4210,  0.3558, -0.6415,  ...,  4.7222, -0.2358,  2.5334],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]]),
 tensor([10,  7], dtype=torch.int32),
 tensor([ 6, 25], dtype=torch.int32))